# Sistema Operativo

## Argomenti

- Lettura e scrittura su file
- Gestione dei percorsi su file system, attraversamento di directory
- Esecuzione di comandi esterni
- Intercettare i segnali

## Lettura e scrittura su file

Elemento centrale di tutte le operazioni di lettura e scrittura su file è il cosiddetto **file object**, o **stream**.

Non è possibile istanziare direttamente un oggetto di tipo stream: per farlo è necessario utilizzare il metodo **open()** della Standard Library.

In [10]:
open("text.txt", 'w')

<_io.TextIOWrapper name='text.txt' mode='w' encoding='cp1252'>

### open(): i parametri

```python
open(file, mode='r', encoding=None, ...)
```

I principali parametri di cui tenere conto sono:

- file: ovvero il percorso
- mode: la modalità di apertura
- encoding: il tipo di [character encoding][2] (solo per file di testo)

[2]: https://en.wikipedia.org/wiki/Character_encoding

### open(): il parametro 'mode'

Una stringa opzionale composta da uno o più dei seguenti caratteri.

* 'r': file in sola lettura
* 'w': file in sola scrittura
* 'a': file in sola scrittura (aggiunge al fondo)
* 'x': sola creazione (fallisce nel caso il file esista già)
* 'b': modalità binaria
* 't': modalità testuale

Se non viene specificato il parametro si assume il valore di **r** (implicitamente **rt**).

### open(): il parametro 'encoding'

Rilevante solo per file aperti in modalità testo (**'t'**).

Se non specificato verrà applicato il valore di default per il sistema operativo.

### close(): l'approccio consigliato (Python >= 2.6)

In [7]:
with open("./text.txt") as stream:
    pass  # Do your things here

Utilizzando il blocco **with .. as ..** non dovremo preoccuparci di gestire la chiusura del file, anche nel caso in cui qualche eccezione venisse sollevata durante l'esecuzione.

### close(): l'approccio classico (Python < 2.6)

In [11]:
try:
    stream = open("./absent.txt")
    # Do your things here
except IOError as ex:
    print("Exception: %s" % ex)
finally:
    stream.close()

Exception: [Errno 2] No such file or directory: './absent.txt'


### File Object API (modalità testo)

- read(): legge il contenuto del file in un'unica chiamata
- write(): scrive il contenuto file
- close(): chiude lo stream

Per la API completa fare riferimento alla documentazione di [IOBase e delle sue sottoclassi][1].

[1]: https://docs.python.org/3/library/io.html#io.IOBase

### Esempi

05.1 Lettura e scrittura su file.ipynb

### Gestione dei percorsi su file system, attraversamento di directory

Esistono varie librerie più o meno di alto livello adatte al compito, le cui principali sono:

- os
- glob
- pathlib

Ci concentreremo su [pathlib][1], l'alternativa più moderna introdotta con la versione 3.4.

E' comunque disponibile un backport per Python >= 2.6 sotto il nome di [pathlib2][2].

[1]: https://docs.python.org/3/library/pathlib.html
[2]: https://pypi.org/project/pathlib2/

### pathlib: importazione

Per prima cosa è necessario importare la classe principale, **Path**:

In [2]:
from pathlib import Path

### pathlib: istanziare la classe Path

Possiamo istanziare il nostro percorso partendo dalla directory di lavoro corrente:

In [39]:
Path.cwd()

WindowsPath('C:/Users/are/Development/python-intro/notebooks')

In [40]:
Path("C:/Users")

WindowsPath('C:/Users')

**nota bene**: sia su Windows che su Linux i percorsi vengono specificati con lo '**/**'. Sarà poi la libreria a prendersi cura di gestire diversamente i percorsi a seconda del sistema operativo sottostante.

### pathlib: istanziare la classe Path con percorsi composti

Una peculiarità di pathlib è il fatto di permettere di specificare percorsi composti in maniera molto intuitiva attraverso l'override dell'operatore '**/**':

In [41]:
Path.cwd() / "00 Struttura del Corso.ipynb"

WindowsPath('C:/Users/are/Development/python-intro/notebooks/00 Struttura del Corso.ipynb')

In [42]:
path = Path("C:/") / "Windows"

### Path: iterdir()

il metodo [iterdir()][1] permette di iterare sul contenuto di una directory.

Se il percorso si riferisce ad un file e non ad una directory verrà sollevata un'eccezione di tipo **NotADirectoryError**: 

[1]: https://docs.python.org/3/library/pathlib.html#pathlib.Path.iterdir

In [64]:
path = Path("C:/Intel")
for file in path.iterdir():
    print(file)

C:\Intel\ExtremeGraphics
C:\Intel\gp
C:\Intel\Logs


### Path: is_file() e is_dir()

In [49]:
Path("C:/Windows").is_dir()

True

In [48]:
path = Path.cwd() / "00 Struttura del Corso.ipynb"
path.is_file()

True

### Path: glob() e il pattern matching

In [63]:
for path in Path.cwd().glob("*Introduzione*"):
    print(path)

C:\Users\are\Development\python-intro\notebooks\01 Introduzione al Linguaggio.ipynb


Anche ricorsivamente con l'uso del pattern '__**__':

In [62]:
for path in Path("C:/Program Files/Git").glob("**/*.pem"):
    print(path)

C:\Program Files\Git\etc\pki\ca-trust\extracted\pem\email-ca-bundle.pem
C:\Program Files\Git\etc\pki\ca-trust\extracted\pem\objsign-ca-bundle.pem
C:\Program Files\Git\etc\pki\ca-trust\extracted\pem\tls-ca-bundle.pem
C:\Program Files\Git\mingw64\etc\pki\ca-trust\extracted\pem\email-ca-bundle.pem
C:\Program Files\Git\mingw64\etc\pki\ca-trust\extracted\pem\objsign-ca-bundle.pem
C:\Program Files\Git\mingw64\etc\pki\ca-trust\extracted\pem\tls-ca-bundle.pem
C:\Program Files\Git\mingw64\ssl\cert.pem
C:\Program Files\Git\usr\ssl\cert.pem


### Path: operare sul contenuto dei file

pathlib mette a disposione anche dei metodi per operare sul contenuto dei file, analoghi a quelli visti in precedenza ma più di alto livello.

I principali sono:

- read_text(): legge l'intero contenuto del file
- write_text(): sovrascrive il contenuto con quello specificato

### Esempi

05.2 Gestione dei percorsi su file system, attraversamento di directory

### Operare su di un file

Pathfile sia un metodo open() con la stessa funzionalità del metodo omonimo della standard library, sia un'API di alto livello per l'accesso al contenuto di singoli file.

Pur non offrendo la stessa flessibilità delle primitive è decisamente comodo per svolgere le operazioni più semplici:

In [49]:
# read_text() API
for file in path.glob("*.md"):
    print(file.read_text())

Una introduzione al linguaggio di programmazione Python tramite notebook Jupyter.


## Eseguire processi esterni

### Il modulo subprocess

**subprocess** è una libreria che permette di eseguire nuovi processi, connettersi alle loro pipe di input/output/error, ed ottenere i loro codici di ritorno (_exit status_).

Rimpiazza varie vecchie librerie, tra le quali:

- os.system
- os.spawn
- os.popen

Per importarla è sufficiente la seguente istruzione:

In [2]:
import subprocess

### subprocess: il metodo run()

Il metodo __run()__ è il punto di ingresso alle funzioni della libreria.

La firma del metodo è piuttosto complessa ma, come vedremo con degli esempi, il suo utilizzo si rivela in realtà piuttosto semplice.

```python
subprocess.run(args, *, stdin=None, input=None, stdout=None, stderr=None, shell=False, cwd=None, timeout=None, check=False, encoding=None, errors=None)
```

### subprocess: il metodo run() - utilizzo base

In [4]:
subprocess.run(["ls", "-l"])

CompletedProcess(args=['ls', '-l'], returncode=0)

[CompletedProcess][1] è un oggetto che rappresenta lo stato dell'esecuzione a posteriori.

Come possiamo vedere da questo esempio ci vengono riportarti due dati:

- il comando che è stato eseguito
- il _returncode_ dell'esecuzione (0=successo, >0=errore)

[1]: https://docs.python.org/3/library/subprocess.html#subprocess.CompletedProcess

### subprocess: il metodo run() - i returncode

Esistono delle convenzioni riguardo ai codici di errore la cui trattazione va però oltre ai nostri scopi. Si vedano:

- [Advanced Bash-Scripting Guide: Appendix E. Exit Codes With Special Meanings][2]
- [man sysexits(3) - FreeBSD Manual Pages][3]


[2]: http://tldp.org/LDP/abs/html/exitcodes.html
[3]: https://www.freebsd.org/cgi/man.cgi?query=sysexits&apropos=0&sektion=0&manpath=FreeBSD%204.3-RELEASE&format=html

### subprocess: il metodo run() - da returncode a eccezione

_"Errors should never pass silently."_ è uno dei principi alla base del linguaggio, in questo caso disatteso dal comportamento di base della libreria.

E' però possibile (e consigliato) forzare il lancio di un'eccezione di tipo [CalledProcessError][1] nel caso in cui il processo esca con un returncode diverso da zero:

[1]: https://docs.python.org/3/library/subprocess.html#subprocess.CalledProcessError

In [69]:
try:
    result = subprocess.run(["ls", "-lz", ".."], check=True)
except subprocess.CalledProcessError as error:
    print('Si è verificato un errore: "{}"'.format(error))

Si è verificato un errore: "Command '['ls', '-lz', '..']' returned non-zero exit status 2."


### subprocess: il metodo run() - catturare stout e stderr

il metodo _run()_ non cattura standard output e standard error di default. Per abilitarne la cattura è necessario passare __PIPE__ come argomento rispettivamente al parametro stdin, a stdout, o ad entrambi:

In [62]:
result = subprocess.run(["ls", "-l", ".."], stdout=subprocess.PIPE)

In [16]:
result.stdout[:100]

b'total 11\n-rw-r--r-- 1 are 1049089  70 Feb 27 14:28 build_pdf.sh\n-rw-r--r-- 1 are 1049089 129 Feb 27 '

### subprocess: il metodo run() - impostare un timeout

Di default _run()_ non impone nessun timeout ai processi eseguiti. Questo spesso non è desiderabile ed è possibile forzare un numero di secondi oltre il quale verrà lanciata un'eccezione di tipo [TimeoutExpired][1] passando un numero come valore al parametro __timeout__:

[1]: https://docs.python.org/3/library/subprocess.html#subprocess.TimeoutExpired

In [19]:
result = subprocess.run(["ls", "-l", ".."], timeout=5, stdout=subprocess.PIPE)

### subprocess: il metodo run() - passaggio dell'input

Per passare dell'input al processo eseguito è necessario passare una stringa come valore del parametro __input__:

In [60]:
input = """Ceci n'est
pas une STDIN"""

subprocess.run(["tac"], stdout=subprocess.PIPE, input=input, encoding="utf-8")

CompletedProcess(args=['tac'], returncode=0, stdout="pas une STDINCeci n'est\n")

## Intercettare i segnali

### Il modulo signal

Tramite la funzione [signal()][1] esportata dal modulo si possono definire delle funzioni che verranno eseguite alla ricezione di un certo segnale (_handler_).

I segnali sono uno strumento di comunicazione asincrona tra processi introdotto nel sistema Unix nel 1970 dai laboratori Bell.

- [man signal][2]
- [man kill][3]

[1]: https://docs.python.org/3/library/signal.html#signal.signal
[2]: http://man7.org/linux/man-pages/man7/signal.7.html
[3]: http://man7.org/linux/man-pages/man1/kill.1.html

### Il modulo signal - i segnali POSIX

- SIGTERM: termina il processo
- SIGINT: interruzione da tastiera (i.e. _CTRL+C_)
- SIGHUP: chiusura del terminale - aggiornamento configurazione
- SIGKILL, SIGSTOP, SIGCNT, etc...

### Il modulo signal - casi d'uso

Nel caso in cui non vengano definiti degli _handler_ i comportamenti di default vengono applicati.

Ciò non è sempre accettabile, ad esempio:

- Pulizia di PID file
- Possibili stati inconsistenti dovuti ad esecuzioni parziali
- I/O flush

### Il modulo signal - definire un handler

In [2]:
import signal
import sys

def handler(signum, frame):
    print('Intercettato segnale', signum)
    sys.exit(0)

signal.signal(signal.SIGTERM, handler)

<function __main__.handler>